<a href="https://colab.research.google.com/github/Chrys97/Hamoye-Stage-B/blob/master/Nwagwu_Chibuikem_Stage_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NWAGWU, CHIBUIKEM CHRYSOGONUS**

importing relevant models and functions

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/My Drive/energydata_complete.csv')
column_names = {'date': 'Date', 'lights': 'Lights', 'T1': 'Kitchen Temperature',
                'RH_1': 'Kitchen Humidity', 'T2': 'Living Room Temperature', 
                'RH_2': 'Living Room Humidity', 'T3': 'Laundry Room Temperature', 
                'RH_3': 'Laundry Room Humidity', 'T4': 'Office Room Temperature', 
                'RH_4': 'Office Room Humidity', 'T5': 'Bathroom Temperature', 
                'RH_5': 'Bathroom Humidity', 'T6': 'Northside Outside Temperature', 
                'RH_6': 'Northside Outside Humidity', 'T7': 'Ironing Room Temperature', 
                'RH_7': 'Ironing Room Humidity', 'T8': 'Teenager Room Temperature', 
                'RH_8': 'Teenager Room Humidity', 'T9': 'Parents Room Temperature', 
                'RH_9': 'Parents Room Humidity', 'To': 'Temperature Outside', 
                'rv1': 'Random Variable 1', 'rv2': 'Random Variable 2'}
df = df.rename(columns = column_names)
df.columns

Index(['Date', 'Appliances', 'Lights', 'Kitchen Temperature',
       'Kitchen Humidity', 'Living Room Temperature', 'Living Room Humidity',
       'Laundry Room Temperature', 'Laundry Room Humidity',
       'Office Room Temperature', 'Office Room Humidity',
       'Bathroom Temperature', 'Bathroom Humidity',
       'Northside Outside Temperature', 'Northside Outside Humidity',
       'Ironing Room Temperature', 'Ironing Room Humidity',
       'Teenager Room Temperature', 'Teenager Room Humidity',
       'Parents Room Temperature', 'Parents Room Humidity', 'T_out',
       'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint',
       'Random Variable 1', 'Random Variable 2'],
      dtype='object')

In [ ]:
df = df.drop(columns = ['Date', 'Lights'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Appliances                     19735 non-null  int64  
 1   Kitchen Temperature            19735 non-null  float64
 2   Kitchen Humidity               19735 non-null  float64
 3   Living Room Temperature        19735 non-null  float64
 4   Living Room Humidity           19735 non-null  float64
 5   Laundry Room Temperature       19735 non-null  float64
 6   Laundry Room Humidity          19735 non-null  float64
 7   Office Room Temperature        19735 non-null  float64
 8   Office Room Humidity           19735 non-null  float64
 9   Bathroom Temperature           19735 non-null  float64
 10  Bathroom Humidity              19735 non-null  float64
 11  Northside Outside Temperature  19735 non-null  float64
 12  Northside Outside Humidity     19735 non-null 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
featured_df = normalized_df.drop(columns = ['Appliances'])
target_variable = normalized_df['Appliances']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(featured_df, target_variable,
                                                    test_size = 0.3,
                                                    random_state = 42)

importing metrics and linear models for use in relevant questions

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [ ]:
models = {'LinearRegression': LinearRegression(),
          'Ridge_reg': Ridge(alpha = 0.4),
          'Lasso_reg': Lasso(alpha = 0.001)}
def pipeline(models, X_train, X_test, y_train, y_test):
  prop = []
  for reg_name, regressor in models.items():
    reg_dict = {}
    reg_dict['Name'] = reg_name
    regressor.fit(x_train, y_train)
    reg_dict['train_score'] = regressor.score(x_train, y_train)
    reg_dict['test_score'] = regressor.score(x_test, y_test)
    reg_dict['MAE'] = mean_absolute_error(y_test, regressor.predict(x_test))
    reg_dict['RMSE'] = np.sqrt(mean_squared_error(y_test, regressor.predict(x_test)))
    reg_dict['RSS'] = np.sum(np.square(y_test - regressor.predict(x_test)))
    reg_dict['R2'] = r2_score(y_test, regressor.predict(x_test))
    prop.append(reg_dict)
    
    summary_df = pd.DataFrame(prop)
  return summary_df
pipeline(models, x_train, x_test, y_train, y_test)

,Name,train_score,test_score,MAE,RMSE,RSS,R2
0,LinearRegression,0.144719,0.148902,0.050134,0.087514,45.347630,0.148902
1,Ridge_reg,0.144506,0.148525,0.050087,0.087534,45.367745,0.148525
2,Lasso_reg,0.024992,0.026801,0.055257,0.093582,51.853367,0.026801


In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.5)
ridge_reg.fit(x_train, y_train)

from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train, y_train)

linear_model_weights = get_weights_df(linear_model, x_train,'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_Weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on ='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on = 'Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_Weight
0,Living Room Humidity,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,Living Room Temperature,-0.236178,-0.193880,0.000000
3,Parents Room Temperature,-0.189941,-0.188584,-0.000000
4,Teenager Room Humidity,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,Ironing Room Humidity,-0.044614,-0.046291,-0.000000
7,Parents Room Humidity,-0.039800,-0.041701,-0.000000
8,Bathroom Temperature,-0.015657,-0.020727,-0.000000
9,Kitchen Temperature,-0.003281,-0.021549,0.000000


**QUESTION 12**

In [ ]:
linear_target = df['Northside Outside Temperature']
array = np.array(df['Living Room Temperature'])
linear_features = array.reshape(-1,1)
x_train, x_test, y_train, y_test = train_test_split(linear_features, linear_target, test_size = 0.3, random_state =42)
simple_linear_model = LinearRegression()
simple_linear_model.fit(x_train, y_train)
predicted_simple_linear = simple_linear_model.predict(x_test)

r2_simple = r2_score(y_test, predicted_simple_linear)
round(r2_simple, 2)

0.64